In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install numpy==1.24.3 librosa ffmpeg-python --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 115.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.24.3 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.24.3 which is incompatible.
albumentations 2.0.5 requires numpy>=1.24.4, but you have numpy 1.24.3 which is incompatible.
blosc2 3.3.0 requires numpy>=1.26, but you have numpy 1.24.3 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.24.3 which is incompatible.
albucore 0.0.23 requires numpy>=1.24.4, but you have numpy 1.24.3 which is incompatible.
treescope 0.1.9 requires numpy>=1.25.2, but you have numpy 1.24.3 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.24.3 which is incompatible.
pymc 5.21.2 requires numpy>=1.25.0,

In [ ]:
!pip install webrtcvad
!apt-get install -y ffmpeg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for webrtcvad: filename=webrtcvad-2.0.10-cp311-cp311-linux_x86_64.whl size=73499 sha256=8910f1fb780fe56a0d5e85ffb017c6de5f86d50b4a4e63352599f876a3bf734d
  Stored in directory: /root/.cache/pip/wheels/94/65/3f/292d0b656be33d1c801831201c74b5f68f41a2ae465ff2ee2f
Successfully built webrtcvad
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [ ]:
import os
import shutil
import wave
import contextlib
import webrtcvad
import collections
from tqdm import tqdm
import numpy as np


video_dir = "/content/drive/MyDrive/urfunny2_videos_copied/urfunny2_videos"
filtered_output_dir = "/content/drive/MyDrive/urfunny2_audio_filtered_videos_vad"
os.makedirs(filtered_output_dir, exist_ok=True)


SPEECH_RATIO_THRESHOLD = 0.85
BATCH_SIZE = 100


def read_wave(path):
    with contextlib.closing(wave.open(path, 'rb')) as wf:
        assert wf.getnchannels() == 1
        assert wf.getsampwidth() == 2
        assert wf.getframerate() in (8000, 16000, 32000, 48000)
        pcm_data = wf.readframes(wf.getnframes())
        return pcm_data, wf.getframerate()

def write_wav(pcm_data, sample_rate, out_path):
    with contextlib.closing(wave.open(out_path, 'wb')) as wf:
        wf.setnchannels(1)
        wf.setsampwidth(2)
        wf.setframerate(sample_rate)
        wf.writeframes(pcm_data)

def extract_audio_ffmpeg(video_path, output_wav_path):
    cmd = f"ffmpeg -y -loglevel error -i '{video_path}' -ar 16000 -ac 1 -vn '{output_wav_path}'"
    return os.system(cmd) == 0

def calculate_vad_ratio(wav_path):
    try:
        audio, sample_rate = read_wave(wav_path)
        vad = webrtcvad.Vad(2)
        frame_duration = 30  # ms
        frame_size = int(sample_rate * frame_duration / 1000) * 2
        frames = [audio[i:i+frame_size] for i in range(0, len(audio), frame_size)]
        voiced = sum(1 for frame in frames if len(frame) == frame_size and vad.is_speech(frame, sample_rate))
        return voiced / len(frames) if frames else 0.0
    except Exception as e:
        print(f"[ERROR] VAD failed for {wav_path}: {e}")
        return 0.0


video_files = sorted([f for f in os.listdir(video_dir) if f.endswith(".mp4")])
total_videos = len(video_files)
print(f" Total videos to process: {total_videos}")

kept_total = 0
dropped_total = 0

for i in range(0, total_videos, BATCH_SIZE):
    batch = video_files[i:i+BATCH_SIZE]
    print(f"\n Processing batch {i//BATCH_SIZE + 1} ({len(batch)} videos)...\n")

    kept_count, dropped_count = 0, 0

    for video_file in tqdm(batch):
        video_path = os.path.join(video_dir, video_file)
        temp_wav = "temp_vad.wav"

        if extract_audio_ffmpeg(video_path, temp_wav):
            ratio = calculate_vad_ratio(temp_wav)
            os.remove(temp_wav)

            if ratio >= SPEECH_RATIO_THRESHOLD:
                shutil.copy(video_path, os.path.join(filtered_output_dir, video_file))
                kept_count += 1
            else:
                dropped_count += 1
        else:
            print(f"[ERROR] Failed to extract audio: {video_file}")
            dropped_count += 1

    kept_total += kept_count
    dropped_total += dropped_count

    print(f" Batch done: Kept {kept_count}, Dropped {dropped_count}")

print("\n All Batches Completed!")
print(f" Total Kept: {kept_total} (speech_ratio >= {SPEECH_RATIO_THRESHOLD})")
print(f" Total Dropped: {dropped_total} (too silent or failed)")


 Total videos to process: 10166

 Processing batch 1 (100 videos)...



100%|██████████| 100/100 [00:58<00:00,  1.72it/s]


 Batch done: Kept 77, Dropped 23

 Processing batch 2 (100 videos)...



  2%|▏         | 2/100 [02:33<2:27:30, 90.31s/it]

[ERROR] Failed to extract audio: 10127.mp4


...

 Batch done: Kept 55, Dropped 45

 Processing batch 101 (100 videos)...



100%|██████████| 100/100 [00:20<00:00,  4.80it/s]


 Batch done: Kept 74, Dropped 26

 Processing batch 102 (66 videos)...



100%|██████████| 66/66 [00:12<00:00,  5.14it/s]

 Batch done: Kept 51, Dropped 15

 All Batches Completed!
 Total Kept: 6560 (speech_ratio >= 0.85)
 Total Dropped: 3606 (too silent or failed)


In [ ]:
import os
import cv2
import shutil
from tqdm import tqdm

video_dir = "/content/drive/MyDrive/urfunny2_audio_filtered_videos_vad"
filtered_output_dir = "/content/drive/MyDrive/urfunny2_filtered_videos_visual_opencv"
os.makedirs(filtered_output_dir, exist_ok=True)

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

def face_detect_ratio(video_path, sample_rate=2, min_ratio=0.6):
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    interval = max(1, int(fps // sample_rate))

    total_sampled = 0
    face_frames = 0
    idx = 0

    while True:
        success, frame = cap.read()
        if not success:
            break
        if idx % interval == 0:
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(gray, 1.3, 5)
            if len(faces) > 0:
                face_frames += 1
            total_sampled += 1
        idx += 1

    cap.release()
    if total_sampled == 0:
        return 0.0
    return face_frames / total_sampled

video_files = sorted([f for f in os.listdir(video_dir) if f.endswith(".mp4")])
print(f" Total videos to process: {len(video_files)}")
kept = 0
dropped = 0

for vid in tqdm(video_files):
    path = os.path.join(video_dir, vid)
    try:
        ratio = face_detect_ratio(path)
        if ratio >= 0.6:
            shutil.copy(path, os.path.join(filtered_output_dir, vid))
            kept += 1
        else:
            dropped += 1
    except Exception as e:
        print(f"[ERROR] {vid}: {e}")
        dropped += 1

print(f"\n Done! Kept: {kept}, Dropped: {dropped}")


 Total videos to process: 6609


100%|██████████| 6609/6609 [3:35:35<00:00,  1.96s/it]


 Done! Kept: 1329, Dropped: 5280


In [ ]:
import cv2
import os
import numpy as np
from tqdm import tqdm
import shutil

video_dir = "/content/drive/MyDrive/urfunny2_filtered_videos"
output_dir = "/content/drive/MyDrive/urfunny2_final_videos_motion_secondwise"
os.makedirs(output_dir, exist_ok=True)

PIXEL_DIFF_THRESHOLD = 5.0
STATIC_RATIO_THRESHOLD = 0.8
MIN_DURATION = 3

def static_video_detect(video_path):
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = total_frames / fps
    if duration < MIN_DURATION:
        cap.release()
        return False

    frame_indices = [int(fps * t) for t in range(int(duration))]
    sampled_frames = []

    for idx in frame_indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        if not ret:
            continue
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        sampled_frames.append(gray)
    cap.release()

    if len(sampled_frames) < 2:
        return True

    diffs = [np.mean(np.abs(sampled_frames[i+1] - sampled_frames[i])) for i in range(len(sampled_frames)-1)]
    static_count = sum(d < PIXEL_DIFF_THRESHOLD for d in diffs)
    static_ratio = static_count / len(diffs)

    return static_ratio >= STATIC_RATIO_THRESHOLD

video_files = sorted([f for f in os.listdir(video_dir) if f.endswith('.mp4')])
kept = 0
dropped = 0

for fname in tqdm(video_files, desc="Second-wise Motion Filtering"):
    fpath = os.path.join(video_dir, fname)
    if static_video_detect(fpath):
        dropped += 1
        print(f"[DROP] {fname} - Too many static seconds")
    else:
        shutil.copy(fpath, os.path.join(output_dir, fname))
        kept += 1

print(f"\n Second-wise filtering completed: Kept {kept}, Dropped {dropped}")


Second-wise Motion Filtering:  15%|█▍        | 194/1329 [01:54<09:45,  1.94it/s]

[DROP] 11561.mp4 - Too many static seconds


Second-wise Motion Filtering:  16%|█▌        | 208/1329 [01:57<03:55,  4.76it/s]

[DROP] 11749.mp4 - Too many static seconds


Second-wise Motion Filtering:  27%|██▋       | 360/1329 [03:13<07:42,  2.09it/s]

...

[DROP] 49.mp4 - Too many static seconds


Second-wise Motion Filtering:  70%|██████▉   | 928/1329 [07:26<04:19,  1.55it/s]

[DROP] 5531.mp4 - Too many static seconds


Second-wise Motion Filtering: 100%|██████████| 1329/1329 [10:33<00:00,  2.10it/s]


 Second-wise filtering completed: Kept 1323, Dropped 6


In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import shutil
import pandas as pd

video_dir = "/content/drive/MyDrive/urfunny2_final_videos_motion_secondwise"
output_dir = "/content/drive/MyDrive/urfunny2_final_videos_no_ppt"
os.makedirs(output_dir, exist_ok=True)


SAMPLE_RATE = 1
PPT_THRESHOLD = 0.3
PIXEL_EDGE_RATIO = 0.02
COLOR_STD_THRESHOLD = 15

def ppt_frame_detect(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150)
    edge_ratio = np.sum(edges > 0) / edges.size
    color_std = np.std(image)
    return edge_ratio < PIXEL_EDGE_RATIO and color_std < COLOR_STD_THRESHOLD

def video_most_ppt(video_path):
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = int(total_frames / fps) if fps > 0 else 0
    sampled_ppt_count = 0
    total_sampled = 0

    for t in range(0, duration, SAMPLE_RATE):
        cap.set(cv2.CAP_PROP_POS_MSEC, t * 1000)
        ret, frame = cap.read()
        if not ret:
            continue
        total_sampled += 1
        if ppt_frame_detect(frame):
            sampled_ppt_count += 1

    cap.release()
    if total_sampled == 0:
        return False
    return (sampled_ppt_count / total_sampled) > PPT_THRESHOLD

video_files = sorted([f for f in os.listdir(video_dir) if f.endswith(".mp4")])
kept, dropped = 0, 0

for fname in tqdm(video_files, desc="PPT Filtering"):
    fpath = os.path.join(video_dir, fname)
    if video_most_ppt(fpath):
        dropped += 1
    else:
        shutil.copy(fpath, os.path.join(output_dir, fname))
        kept += 1

summary_df = pd.DataFrame({
    "Total Videos": [len(video_files)],
    "Kept": [kept],
    "Dropped (PPT Dominated)": [dropped]
})
print(summary_df)

PPT Filtering: 100%|██████████| 1323/1323 [10:06<00:00,  2.18it/s]

   Total Videos  Kept  Dropped (PPT Dominated)
0          1323  1322                        1


In [ ]:
import os

final_video_dir = "/content/drive/MyDrive/urfunny2_final_videos_no_ppt"
video_files = [f for f in os.listdir(final_video_dir) if f.endswith(".mp4")]
print(f" Total videos remaining after all filters: {len(video_files)}")


 Total videos remaining after all filters: 1322
